In [1]:
# быстрые токенайзеры
!pip install youtokentome

     |████████████████████████████████| 1.7MB 3.4MB/s 


In [0]:
import youtokentome as yttm
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
import numpy as np
import os
import re

import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch import nn
from torch import optim
import torch.nn.functional as F

from tqdm import tqdm_notebook as tqdm
import utils

Я достаточно долго пытался понять, как было произведено трейн/тест деление. Так как пересечение уникальных русских/английских названий компаний между трейном и тестом очень маленькое (всего 13% русских и 5% английских названий из теста встречаются в трейне), то это явно не случайное (равномерное) разделение.

Я сразу подумал о представлении данных в виде двудольного графа: русские имена и английские имена. Если между ними есть ребро, то такая пара в данных встречается, иначе ребра нет. Я пытался загуглить задачу, в которой вершины двудольного графа делят на два непепресекающихся подмножества так, что минимизируется количество ребер между вершинами из разных подмножеств. Все, что я нашел – это один [вопрос](https://math.stackexchange.com/questions/2637808/bipartite-graph-partitioning-special-case?rq=1) на math.stackexchange с задачей с похожей формулировкой, но без ответа. Отсюда можно сделать вывод, что вы не решали ничего похожего.

Тогда, по-моему, единственное объяснение – это то, что у вас было больше данных, но некоторую часть ребер (таких, что какая-то его вершина встречаются и в трейне и в тесте) вы удалили, чтобы сделать валидацию более надежной.

Так как при случайном разделении вышеупомянутые 13% и 5% превращаются в 90+%, то урезание до нужных процентов привело бы к тому, что данных стало бы заметно меньше. К тому же, я использую два валидационных множества: одно для нахождения границы предсказывания, а другое – для подсчитывания F1 метрики, что также рождает трудности: непонятно, как сделать так, чтобы эти 13 и 5 процентов были как между трейном и первым валидационным множеством, так и между трейном и вторым валидационным множеством, так и между первым и вторым валидационными множествами.

Поэтому я принял решение сфокусироваться на моделинге и валидировать на множестве с т.н. "ликами" :) Так как пересечение по английским названиям меньше, то я решил делать GroupShuffleSplit по колонке с английскими названиями, т.е. не ликать английские названия в валидационное множество.

In [0]:
SEED = 42

In [0]:
train = pd.read_csv('kontur_srs_test_task/train_data.tsv', sep='\t')
train, thresh_split, holdout_split = utils.split_train(train, 'eng_name', thresh_size=0.4, seed=SEED)

В качестве модели я использую сиамские сверточные сети с max-пулингом на BPE-токенах, полученных с очищенных данных. Я использую majority-vote ансамбль 3 моделей, натренированных на данных с разными сидами, чтобы задействовать все тренировочные данные. Данные к нижнему регистру я не приводил, так как качество предсказаний после потери регистра заметно падало. Вдохновение я взял с [SentenceBert](https://arxiv.org/abs/1908.10084): там два предложения энкодятся (с mean-пулингом) одним и тем же БЕРТом в вектора u и v, и по конкатенации векторов u, v и |u-v| полносвязный классификатор определяет степень похожести предложений (в моем случае – это бинарный классификатор). Затем на инференсе в качестве показателя похожести предложений берется косинусовая похожесть эмбеддингов предложений.

Я быстро определил, что mean-пулинг нужно поменять на max-, и затем изучал влияние гиперпараметров на результат работы модели. Изначально я хотел сделать ансамбль из сиамских ЛСТМок, сиамских трансформеров (ха-ха) и сиамских сверток, но в итоге ЛСТМки показали себя хуже сверток, а смысла в ансамбле сильно отличающихся по качеству моделей нет. Трансформер (энкодер, разумеется), увы, просто отказался учить хоть что-то полезное :(

Также я хотел добавить аугментацию данных: например, поиграться с регистрами слов, но не успел по времени.

Итак, моя модель – это:


*   Эмбеддинг BPE-токенов в 175-мерное пространство (200 для одной из моеделей);
*   3 свертки, не меняющих размерность признаков, с окном в 5 и с паддингом;
*   ReLu после первой и второй свертки;
*   Dropout между первым ReLu и второй сверткой, он же между вторым ReLu и третьей сверткой;
*   tanh после третьей свертки для нормализации активаций, затем – глобальный макспулинг;
*   через такой пайплайн прогоняются два названия – русское и английское. Пусть их вектора будут u и v соответственно. Конкатенация векторов u, v и |u-v| пропускается через дропаут и подается на вход однослойному бинарному классификатору;
*   инференс – получаются вектора u и v, и считается их косинусовая схожесть. Затем по первому валидационному множеству находится трешхолд, который максимизирует F1 метрику, и он же применяется ко второму валидационному множеству. Ну или к тестовому. Все :)



In [0]:
class SiameseCNNEncoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dims, kernel_sizes, paddings, dropout):
        super(SiameseCNNEncoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=utils.PAD_ID)

        convolutional_layers = []
        for i in range(len(paddings)):
          if i != 0:
            convolutional_layers.append(nn.Dropout(dropout))
          convolutional_layers.append(nn.Conv1d(hidden_dims[i], 
                                        hidden_dims[i + 1], 
                                        kernel_sizes[i], 
                                        padding=paddings[i]))
          if i != len(paddings) - 1:
            convolutional_layers.append(nn.ReLU())
        
        self.convolutions = nn.Sequential(*convolutional_layers)
        self.dropout_final = nn.Dropout(dropout)
        self.linear = nn.Linear(3 * hidden_dims[-1], 2)
        
    def _embed(self, x):
        x = self.embedding(x)
        x = x.transpose(1, 2)
        x = self.convolutions(x)
        x = torch.tanh(x)
        x, _ = x.max(dim=2)
        return x
        
    def forward(self, x, y):
        x = self._embed(x)
        y = self._embed(y)
        embedding = torch.cat([x, y, torch.abs(x - y)], axis=-1)
        embedding = self.dropout_final(embedding)
        logits = self.linear(embedding)
        return logits
    
    def compute_similarity(self, x, y):
        x = self._embed(x)
        y = self._embed(y)
        cosine_similarity = F.cosine_similarity(x, y)
        return cosine_similarity

# Обучение 
(не обязательно запускать для того, чтобы работал инференс, который ниже)

In [0]:
EPOCHS = 15
BATCH_SIZE = 200

VOCAB_SIZE = 320
EMBEDDING_DIM = 175
HIDDEN_DIMS = [EMBEDDING_DIM, EMBEDDING_DIM, EMBEDDING_DIM, EMBEDDING_DIM]
KERNEL_SIZES = [5, 5, 5]
DROPOUT = 0.15
PADDINGS = [2, 2, 2]

LR = 0.01

tokenizer_filename = 'tokenizer_320cased_1.model'

Чистка данных: привожу некоторые символы к символам, которые они должны обозначать, а также убираю некоторые символы (те, что не из вайтлиста в регексе).

In [0]:
def transform(string):
  rep = {'<<': '"', 
         '>>': '"',
         '<': '"', 
         '>': '"', 
         "''": '"', 
         '``': '"',
         '`': "'", 
         '/': ' ', 
         '«': '"', 
         '»': '"',
         '–': '-',
         '_': '-'}
  rep = dict((re.escape(k), v) for k, v in rep.items()) 

  pattern = re.compile("|".join(rep.keys()))
  replaced = pattern.sub(lambda m: rep[re.escape(m.group(0))], string)
  filtered = ''.join(re.findall("[A-Za-z0-9а-яА-Я\-',.\(\)\" &@!?\+\*№]*", replaced))
  if filtered.strip() == '':
    # empty string token
    filtered = '#'
  return filtered

In [0]:
train = utils.transform_dataframe(train, transform)
thresh_split = utils.transform_dataframe(thresh_split, transform)
holdout_split = utils.transform_dataframe(holdout_split, transform)

In [0]:
utils.train_bpe_tokenizer(train, tokenizer_filename, VOCAB_SIZE)
tokenizer = yttm.BPE(model=tokenizer_filename)

Заранее перевожу строковые данные в id токенов, чтобы не тратить время на токенизацию во время обучения.

In [0]:
train_lists = utils.precompute_dataset(train, tokenizer)

threshold_lists = utils.precompute_dataset(thresh_split, tokenizer)
holdout_lists = utils.precompute_dataset(holdout_split, tokenizer)

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [39]:
model = SiameseCNNEncoder(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIMS, KERNEL_SIZES, PADDINGS, DROPOUT)
model.to(device)

SiameseCNNEncoder(
  (embedding): Embedding(320, 175, padding_idx=0)
  (convolutions): Sequential(
    (0): Conv1d(175, 175, kernel_size=(5,), stride=(1,), padding=(2,))
    (1): ReLU()
    (2): Dropout(p=0.15, inplace=False)
    (3): Conv1d(175, 175, kernel_size=(5,), stride=(1,), padding=(2,))
    (4): ReLU()
    (5): Dropout(p=0.15, inplace=False)
    (6): Conv1d(175, 175, kernel_size=(5,), stride=(1,), padding=(2,))
  )
  (dropout_final): Dropout(p=0.15, inplace=False)
  (linear): Linear(in_features=525, out_features=2, bias=True)
)

In [0]:
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, nesterov=True)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=1, factor=0.2, verbose=True, mode='max')
criterion = nn.CrossEntropyLoss()

In [0]:
train_loader = DataLoader(utils.ListsDataset(*train_lists), shuffle=True, batch_size=BATCH_SIZE, collate_fn=utils.collate_fn)

threshold_loader = DataLoader(utils.ListsDataset(*threshold_lists), batch_size=BATCH_SIZE, collate_fn=utils.collate_fn)
holdout_loader = DataLoader(utils.ListsDataset(*holdout_lists), batch_size=BATCH_SIZE, collate_fn=utils.collate_fn)

In [0]:
# для нахождения границы предсказывания
threshold_tuner = utils.OptimizedRounder(0.73)

In [0]:
best_preds = None
best_score = 0.0

EPOCHS_WITHOUT_VALID = 6

In [0]:
# для воспроизводимости
torch.manual_seed(42)
np.random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [0]:
for epoch in range(1, EPOCHS + 1):
  model.train()
  for i, batch in enumerate(tqdm(train_loader, desc='Epoch {}: '.format(epoch))):
    ru, eng, labels = batch
    ru, eng, labels = ru.to(device), eng.to(device), labels.to(device)
    pred = model(ru, eng)
    loss = criterion(pred, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if epoch > EPOCHS_WITHOUT_VALID:
    similarities = []
    trues = []

    model.eval()
    with torch.no_grad():
      for batch in threshold_loader:
        ru, eng, labels = batch
        trues.extend(list(labels.numpy()))
        ru, eng = ru.to(device), eng.to(device)
        similarities.extend(list(model.compute_similarity(ru, eng).cpu().numpy()))

    threshold_tuner.fit(similarities, trues)

    similarities = []
    trues = []

    with torch.no_grad():
      for batch in holdout_loader:
        ru, eng, labels = batch
        trues.extend(list(labels.numpy()))
        ru, eng = ru.to(device), eng.to(device)
        similarities.extend(list(model.compute_similarity(ru, eng).cpu().numpy()))

    def compute_score(tuner):
      preds = tuner.predict(similarities)
      prec = utils.precision(preds, trues)
      rec = utils.recall(preds, trues)
      f1 = utils.f_score(preds, trues)
      print('Threshold: ', tuner.thresh_) 
      print('Precision: ', prec)
      print('Recall: ', rec)
      print('F1 score: ', f1)
      return f1

    score = compute_score(threshold_tuner)
    if score > best_score:
      torch.save({
        'epoch': epoch,
        'score': score,
        'seed': SEED,
        'vocab_Size': VOCAB_SIZE,
        'dropout': DROPOUT,
        'embedding_dim': EMBEDDING_DIM,
        'hidden_dims': HIDDEN_DIMS,
        'paddings': PADDINGS,
        'kernel_sizes': KERNEL_SIZES,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()}, 
        'checkpoint_cnn_1')
      best_score = score
      
    scheduler.step(score)

# Инференс (все 3 модели)
(около 8 минут на колабе)

In [0]:
train = pd.read_csv('kontur_srs_test_task/train_data.tsv', sep='\t')
test = pd.read_csv('kontur_srs_test_task/test_data.tsv', sep='\t')

def transform(string):
  rep = {'<<': '"', 
         '>>': '"',
         '<': '"', 
         '>': '"', 
         "''": '"', 
         '``': '"',
         '`': "'", 
         '/': ' ', 
         '«': '"', 
         '»': '"',
         '–': '-',
         '_': '-'}
  rep = dict((re.escape(k), v) for k, v in rep.items()) 

  pattern = re.compile("|".join(rep.keys()))
  replaced = pattern.sub(lambda m: rep[re.escape(m.group(0))], string)
  filtered = ''.join(re.findall("[A-Za-z0-9а-яА-Я\-',.\(\)\" &@!?\+\*№]*", replaced))
  if filtered.strip() == '':
    # empty string token
    filtered = '#'
  return filtered

test_transformed = utils.transform_dataframe(test, transform)

all_preds = np.zeros((len(test), 3))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for i in range(1, 4):
  tokenizer = yttm.BPE(model='models/tokenizer_320cased_{}.model'.format(i))
  cp = torch.load('models/checkpoint_cnn_{}'.format(i))

  SEED = cp['seed']
  VOCAB_SIZE = cp['vocab_size']
  EMBEDDING_DIM = cp['embedding_dim']
  HIDDEN_DIMS = cp['hidden_dims']
  KERNEL_SIZES = cp['kernel_sizes']
  PADDINGS = cp['paddings']
  DROPOUT = cp['dropout']

  _, thresh_split, _ = utils.split_train(train, 'eng_name', thresh_size=0.4, seed=SEED)

  thresh_split = utils.transform_dataframe(thresh_split, transform)
  threshold_lists = utils.precompute_dataset(thresh_split, tokenizer)
  test_lists = utils.precompute_dataset(test_transformed, tokenizer)

  threshold_loader = DataLoader(utils.ListsDataset(*threshold_lists), batch_size=200, collate_fn=utils.collate_fn)
  test_loader = DataLoader(utils.ListsDataset(*test_lists), batch_size=200, collate_fn=lambda batch: utils.collate_fn(batch, with_labels=False))

  model = SiameseCNNEncoder(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIMS, KERNEL_SIZES, PADDINGS, DROPOUT)
  model.to(device)

  model.load_state_dict(cp['model_state_dict'])

  threshold_tuner = utils.OptimizedRounder(0.73)

  similarities = []
  trues = []

  model.eval()
  with torch.no_grad():
    for batch in threshold_loader:
      ru, eng, labels = batch
      trues.extend(list(labels.numpy()))
      ru, eng = ru.to(device), eng.to(device)
      similarities.extend(list(model.compute_similarity(ru, eng).cpu().numpy()))

  threshold_tuner.fit(similarities, trues)

  similarities = []

  with torch.no_grad():
    for batch in test_loader:
      ru, eng = batch
      ru, eng = ru.to(device), eng.to(device)
      similarities.extend(list(model.compute_similarity(ru, eng).cpu().numpy()))
  
  preds = threshold_tuner.predict(similarities)
  all_preds[:, i - 1] = preds


final_preds = np.apply_along_axis(lambda preds: np.median(preds).astype(bool), 1, all_preds)


In [0]:
test['answer'] = final_preds
test.to_csv('answers.tsv', sep='\t')